# Working notebook for clustering project

In [1]:

#Basic libraries
import pandas as pd
import numpy as np 
import numpy as np

#Vizualization Tools
import matplotlib.pyplot as plt
import seaborn as sns

#Modeling Tools
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import statsmodels.api as sm

from datetime import date

import sklearn.preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


import warnings
warnings.filterwarnings("ignore")

#Custim functions
from env import host, user, password #Database credentials
import wrangle
import wrangle2


In [2]:
## URL from ENV
def get_db_url(database):
    '''
    Gets appropriate url to pull data from credentials stored in env file
    '''
    from env import host, user, password
    url = f'mysql+pymysql://{user}:{password}@{host}/{database}'
    return url


In [3]:
# Query

def get_zillow():
    query = '''
    SELECT prop.*, 
        pred.logerror, 
        pred.transactiondate, 
        air.airconditioningdesc, 
        arch.architecturalstyledesc, 
        build.buildingclassdesc, 
        heat.heatingorsystemdesc, 
        landuse.propertylandusedesc, 
        story.storydesc, 
        construct.typeconstructiondesc 

    FROM   properties_2017 prop  
        INNER JOIN (SELECT parcelid,
                            logerror,
                            Max(transactiondate) transactiondate 
                    FROM   predictions_2017 
                    GROUP  BY parcelid, logerror) pred
                USING (parcelid) 
        LEFT JOIN airconditioningtype air USING (airconditioningtypeid) 
        LEFT JOIN architecturalstyletype arch USING (architecturalstyletypeid) 
        LEFT JOIN buildingclasstype build USING (buildingclasstypeid) 
        LEFT JOIN heatingorsystemtype heat USING (heatingorsystemtypeid) 
        LEFT JOIN propertylandusetype landuse USING (propertylandusetypeid) 
        LEFT JOIN storytype story USING (storytypeid) 
        LEFT JOIN typeconstructiontype construct USING (typeconstructiontypeid) 
    WHERE  prop.latitude IS NOT NULL 
        AND prop.longitude IS NOT NULL AND transactiondate <= '2017-12-31' 
    '''

    df = pd.read_sql(query, get_db_url('zillow'), index_col='id')
    return df


In [ ]:
#Pull values from SQL

#df = get_zillow()

In [ ]:
#df.head()

In [ ]:
#Write dataframe to csv
#df.to_csv('unedited_zillow.csv')

In [4]:
# Read CSV
df =pd.read_csv('unedited_zillow.csv')

In [5]:
df.head()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,1727539,14297519,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,...,6.059063e+13,0.025595,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
1,1387261,17052889,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,...,6.111001e+13,0.055619,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,11677,14186244,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,...,6.059022e+13,0.005383,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
3,2288172,12177905,NaN,NaN,NaN,3.0,4.0,NaN,8.0,3.0,...,6.037300e+13,-0.103410,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
4,1970746,10887214,1.0,NaN,NaN,3.0,3.0,NaN,8.0,3.0,...,6.037124e+13,0.006940,2017-01-01,Central,NaN,NaN,Central,Condominium,NaN,NaN


In [6]:
shape1 = df.shape
shape1

(77574, 68)

In [7]:
#Ensure single use homes

def single_use(df):
    '''
    Ensures we are only looking at single use properties with at least one bedroom and >= 350 sf.
    '''
    single_use = [261, 262, 263, 264, 266, 268, 273, 276, 279]
    df = df[df.propertylandusetypeid.isin(single_use)]
    df = df[(df.bedroomcnt > 0) & (df.bathroomcnt > 0) & ((df.unitcnt<=1)|df.unitcnt.isnull())\
            & (df.calculatedfinishedsquarefeet>350)]
    return df

In [8]:
df=single_use(df)

In [9]:
shape2 = df.shape
print(shape1)
print(shape2)
print(f'Dropped {shape1[0]-shape2[0]} rows')

(77574, 68)
(71355, 68)
Dropped 6219 rows


In [10]:
def add_county(df):
    '''
    Add column for counties
    '''
    import numpy as np
    df['county'] = np.where(df.fips == 6037, 'Los_Angeles',
                           np.where(df.fips == 6059, 'Orange', 
                                   'Ventura'))    
    return df

In [11]:
df = add_county(df)

In [12]:
df.head()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc,county
0,1727539,14297519,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,...,0.025595,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN,Orange
1,1387261,17052889,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,...,0.055619,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN,Ventura
2,11677,14186244,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,...,0.005383,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN,Orange
3,2288172,12177905,NaN,NaN,NaN,3.0,4.0,NaN,8.0,3.0,...,-0.103410,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN,Los_Angeles
4,1970746,10887214,1.0,NaN,NaN,3.0,3.0,NaN,8.0,3.0,...,0.006940,2017-01-01,Central,NaN,NaN,Central,Condominium,NaN,NaN,Los_Angeles


In [13]:
def handle_missing_values(df, prop_required_column = .5, prop_required_row = .70):
    threshold = int(round(prop_required_column*len(df.index),0))
    df.dropna(axis=1, thresh=threshold, inplace=True)
    threshold = int(round(prop_required_row*len(df.columns),0))
    df.dropna(axis=0, thresh=threshold, inplace=True)
    return df

In [ ]:
df.shape

In [14]:
df = handle_missing_values(df)

In [15]:
shape3=df.shape
print(shape3)
print(f'Dropped {shape2[0]-shape3[0]} rows')
print(f'Dropped {shape2[1]-shape3[1]} columns')

(71355, 35)
Dropped 0 rows
Dropped 33 columns


In [16]:
def remove_columns(df, cols_to_remove):  
    '''
    Pass a list od columns to remove
    '''
    df = df.drop(columns=cols_to_remove)
    return df

In [17]:
def clean(df):    
    # replace nulls with median values for select columns
    df.lotsizesquarefeet.fillna(7313, inplace = True)
    # Columns to look for outliers
    df = df[df.taxvaluedollarcnt < 5_000_000]
    df[df.calculatedfinishedsquarefeet < 8000]
    # Just to be sure we caught all nulls, drop them here
    df = df.dropna()
    return df

In [18]:
df=clean(df)

In [ ]:
shape4=df.shape
print(shape4)
print(f'Dropped {shape3[0]-shape4[0]} rows')
print(f'Dropped {shape3[1]-shape4[1]} columns')

In [19]:
def remove_outliers(df, col_list, k=1.5):
    ''' remove outliers from a list of columns in a dataframe 
        and return that dataframe
    '''
    for col in col_list:
        q1, q3 = df[col].quantile([.25, .75])  # get quartiles
        iqr = q3 - q1   # calculate interquartile range
        upper_bound = q3 + k * iqr   # get upper bound
        lower_bound = q1 - k * iqr   # get lower bound
        # return dataframe without outliers
        df = df[(df[col] > lower_bound) & (df[col] < upper_bound)]
    return df

In [ ]:
columns=['bathroomcnt']

- [ ] Throw out outliers in target?
- [ ] What about year?

In [ ]:
df = remove_outliers(df, ['bathroomcnt', 'bedroomcnt', 'calculatedbathnbr', 'calculatedfinishedsquarefeet', 'finishedsquarefeet12', 'fullbathcnt', 'lotsizesquarefeet', 'roomcnt', 'unitcnt','structuretaxvaluedollarcnt', 'taxvaluedollarcnt','taxamount'], 1.5)

In [ ]:
shape5=df.shape
print(shape5)
print(f'Dropped {shape4[0]-shape5[0]} rows')
print(f'Dropped {shape4[1]-shape5[1]} columns')

In [20]:
df.dtypes

id                                int64
parcelid                          int64
bathroomcnt                     float64
bedroomcnt                      float64
buildingqualitytypeid           float64
calculatedbathnbr               float64
calculatedfinishedsquarefeet    float64
finishedsquarefeet12            float64
fips                            float64
fullbathcnt                     float64
heatingorsystemtypeid           float64
latitude                        float64
longitude                       float64
lotsizesquarefeet               float64
propertycountylandusecode        object
propertylandusetypeid           float64
propertyzoningdesc               object
rawcensustractandblock          float64
regionidcity                    float64
regionidcounty                  float64
regionidzip                     float64
roomcnt                         float64
unitcnt                         float64
yearbuilt                       float64
structuretaxvaluedollarcnt      float64


In [21]:
df.shape

(42767, 35)

In [22]:
def split_my_data(df, pct=0.10):
    '''
    This splits a dataframe into train, validate, and test sets. 
    df = dataframe to split
    pct = size of the test set, 1/2 of size of the validate set
    Returns three dataframes (train, validate, test)
    '''
    train_validate, test = train_test_split(df, test_size=pct, random_state = 123)
    train, validate = train_test_split(train_validate, test_size=pct*2, random_state = 123)
    return train, validate, test

In [23]:
train, validate, test = split_my_data(df)


In [24]:
print(train.shape)
print(validate.shape)
print(test.shape)

(30792, 35)
(7698, 35)
(4277, 35)


In [25]:
def min_max_scaler(train, validate, test):
    '''
    Uses the train & test datasets created by the split_my_data function
    Returns 3 items: mm_scaler, train_scaled_mm, test_scaled_mm
    This is a linear transformation. Values will lie between 0 and 1
    '''
    num_vars = list(train.select_dtypes('number').columns)
    scaler = MinMaxScaler(copy=True, feature_range=(0,1))
    train[num_vars] = scaler.fit_transform(train[num_vars])
    validate[num_vars] = scaler.transform(validate[num_vars])
    test[num_vars] = scaler.transform(test[num_vars])
    return scaler, train, validate, test

In [ ]:
train.head()

In [26]:
def add_baseline(train, validate, test):
    '''
    Assigns mean error as baseline prediction
    '''
    baseline = train.logerror.mean()
    train['baseline'] = baseline
    validate['baseline'] = baseline
    test['baseline'] = baseline
    return train, validate, test

In [27]:
train, validate, test = add_baseline(train, validate, test)

In [28]:
train.head()

,id,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,...,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,logerror,transactiondate,heatingorsystemdesc,propertylandusedesc,county,baseline
73629,2253340,10749301,1.0,1.0,8.0,1.0,910.0,910.0,6037.0,1.0,...,2016.0,106200.0,3341.76,6.037800e+13,0.148054,2017-09-05,Central,Condominium,Los_Angeles,0.014418
54594,2594283,11664530,5.0,5.0,12.0,5.0,4058.0,4058.0,6037.0,5.0,...,2016.0,2117300.0,44590.86,6.037263e+13,0.020376,2017-07-06,Central,Single Family Residential,Los_Angeles,0.014418
24158,1227560,11698152,1.0,2.0,6.0,1.0,1020.0,1020.0,6037.0,1.0,...,2016.0,226803.0,3879.70,6.037703e+13,-0.045610,2017-04-06,Floor/Wall,Single Family Residential,Los_Angeles,0.014418
21910,1509888,11390415,1.0,2.0,4.0,1.0,1086.0,1086.0,6037.0,1.0,...,2016.0,251320.0,3931.92,6.037235e+13,0.032872,2017-03-30,Floor/Wall,Single Family Residential,Los_Angeles,0.014418
66558,960749,12130057,3.0,3.0,7.0,3.0,2434.0,2434.0,6037.0,3.0,...,2016.0,60449.0,2683.37,6.037430e+13,0.041174,2017-08-15,Central,Single Family Residential,Los_Angeles,0.014418


In [29]:
from sklearn.preprocessing import MinMaxScaler

In [30]:
train.head()

,id,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,...,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,logerror,transactiondate,heatingorsystemdesc,propertylandusedesc,county,baseline
73629,2253340,10749301,1.0,1.0,8.0,1.0,910.0,910.0,6037.0,1.0,...,2016.0,106200.0,3341.76,6.037800e+13,0.148054,2017-09-05,Central,Condominium,Los_Angeles,0.014418
54594,2594283,11664530,5.0,5.0,12.0,5.0,4058.0,4058.0,6037.0,5.0,...,2016.0,2117300.0,44590.86,6.037263e+13,0.020376,2017-07-06,Central,Single Family Residential,Los_Angeles,0.014418
24158,1227560,11698152,1.0,2.0,6.0,1.0,1020.0,1020.0,6037.0,1.0,...,2016.0,226803.0,3879.70,6.037703e+13,-0.045610,2017-04-06,Floor/Wall,Single Family Residential,Los_Angeles,0.014418
21910,1509888,11390415,1.0,2.0,4.0,1.0,1086.0,1086.0,6037.0,1.0,...,2016.0,251320.0,3931.92,6.037235e+13,0.032872,2017-03-30,Floor/Wall,Single Family Residential,Los_Angeles,0.014418
66558,960749,12130057,3.0,3.0,7.0,3.0,2434.0,2434.0,6037.0,3.0,...,2016.0,60449.0,2683.37,6.037430e+13,0.041174,2017-08-15,Central,Single Family Residential,Los_Angeles,0.014418


In [31]:
def split_xy(train, validate, test):
    '''
    Splits dataframe into train, validate, and test data frames
    '''
    X_train = train.drop(columns='logerror')
    y_train = train.logerror

    X_validate = validate.drop(columns='logerror')
    y_validate = validate.logerror

    X_test = test.drop(columns='logerror')
    y_test = test.logerror

    return train, X_train, y_train, X_validate, y_validate, X_test, y_test

In [32]:
train, X_train, y_train, X_validate, y_validate, X_test, y_test = split_xy(train, validate, test)

In [33]:
def scale(X_train, X_validate, X_test, train, validate, test):
    '''
    Uses the train & test datasets created by the split_my_data function
    Returns 3 items: mm_scaler, train_scaled_mm, test_scaled_mm
    This is a linear transformation. Values will lie between 0 and 1
    '''
    num_vars = list(X_train.select_dtypes('number').columns)
    scaler = MinMaxScaler(copy=True, feature_range=(0,1))
    train[num_vars] = scaler.fit_transform(X_train[num_vars])
    validate[num_vars] = scaler.transform(X_validate[num_vars])
    test[num_vars] = scaler.transform(X_test[num_vars])
    return X_train, X_validate, X_test

In [34]:
X_train, X_validate, X_test = scale(X_train, X_validate, X_test, train, validate, test)

In [35]:
train.shape

(30792, 36)

In [36]:
def wrangle():
    df = pd.read_csv('unedited_zillow.csv')
    df = single_use(df)
    df = add_county(df)
    df = handle_missing_values(df)
    df = clean(df)
    train, validate, test = split_my_data(df)
    train, validate, test = add_baseline(train, validate, test)
    train, X_train, y_train, X_validate, y_validate, X_test, y_test = split_xy(
        train, validate, test)
    X_train, X_validate, X_test = scale(
        X_train, X_validate, X_test, train, validate, test)
    return train, X_train, y_train, X_validate, y_validate, X_test, y_test

In [37]:
train, X_train, y_train, X_validate, y_validate, X_test, y_test=wrangle()

In [38]:
X_train.describe()

,id,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,...,roomcnt,unitcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,baseline
count,3.079200e+04,3.079200e+04,30792.000000,30792.000000,30792.000000,30792.000000,30792.000000,30792.000000,30792.0,30792.000000,...,30792.0,30792.0,30792.000000,3.079200e+04,3.079200e+04,30792.0,3.079200e+04,30792.000000,3.079200e+04,3.079200e+04
mean,1.500702e+06,1.185344e+07,2.197876,2.957814,6.581645,2.197876,1662.633054,1662.633054,6037.0,2.197876,...,0.0,1.0,1963.376299,1.733416e+05,4.406668e+05,2016.0,2.673252e+05,5541.574264,6.038816e+13,1.441769e-02
std,8.600178e+05,7.224266e+05,0.940180,0.973725,1.677139,0.940180,825.816224,825.816224,0.0,0.940180,...,0.0,0.0,23.247708,1.755103e+05,4.678705e+05,0.0,3.351853e+05,5498.919327,2.408619e+12,5.096700e-15
min,1.307000e+03,1.071186e+07,1.000000,1.000000,1.000000,1.000000,360.000000,360.000000,6037.0,1.000000,...,0.0,1.0,1878.000000,1.810000e+02,3.254000e+03,2016.0,1.393000e+03,120.840000,6.037101e+13,1.441769e-02
25%,7.647012e+05,1.118624e+07,2.000000,2.000000,6.000000,2.000000,1136.000000,1136.000000,6037.0,2.000000,...,0.0,1.0,1950.000000,8.200000e+04,1.786135e+05,2016.0,6.072075e+04,2497.760000,6.037218e+13,1.441769e-02
50%,1.508579e+06,1.185516e+07,2.000000,3.000000,6.000000,2.000000,1452.000000,1452.000000,6037.0,2.000000,...,0.0,1.0,1960.000000,1.305350e+05,3.183100e+05,2016.0,1.727220e+05,4150.670000,6.037430e+13,1.441769e-02
75%,2.251414e+06,1.251059e+07,3.000000,4.000000,8.000000,3.000000,1937.000000,1937.000000,6037.0,3.000000,...,0.0,1.0,1981.000000,2.016900e+05,5.216940e+05,2016.0,3.365732e+05,6461.642500,6.037577e+13,1.441769e-02
max,2.982188e+06,1.310220e+07,10.000000,11.000000,12.000000,10.000000,12039.000000,12039.000000,6037.0,10.000000,...,0.0,1.0,2016.000000,2.860000e+06,4.984456e+06,2016.0,4.122282e+06,61992.630000,4.830301e+14,1.441769e-02


In [ ]:
## Wrangle function
def wrangle():
    #df = get_zillow()
    #df.to_csv('unedited_zillow.csv')
    df =pd.read_csv('unedited_zillow.csv')
    df = single_use(df)
    df = add_county(df)
    df = handle_missing_values(df)
    df = clean(df)
    #columns=['bathroomcnt', 'bedroomcnt', 'calculatedbathnbr', 'calculatedfinishedsquarefeet', 'finishedsquarefeet12', 'fullbathcnt', 'lotsizesquarefeet', 'roomcnt', 'unitcnt','structuretaxvaluedollarcnt', 'taxvaluedollarcnt','taxamount']
    #df = remove_outliers(df, columns)
    train, validate, test = split_my_data(df)
    train, validate, test = add_baseline(train, validate, test)
    train, X_train, y_train, X_validate, y_validate, X_test, y_test = split_xy(train, validate, test)
    X_train, X_validate, X_test = scale(X_train, X_validate, X_test, train, validate, test)
    return train, X_train, y_train, X_validate, y_validate, X_test, y_test


In [ ]:
def wrangle():
    df = pd.read_csv('unedited_zillow.csv')
    df = single_use(df)
    df = add_county(df)
    df = handle_missing_values(df)
    df = clean(df)
    train, validate, test = split_my_data(df)
    train, validate, test = add_baseline(train, validate, test)
    train, X_train, y_train, X_validate, y_validate, X_test, y_test = split_xy(
        train, validate, test)
    X_train, X_validate, X_test = scale(
        X_train, X_validate, X_test, train, validate, test)
    return X_train, X_validate, X_test

In [ ]:
train, X_train, y_train, X_validate, y_validate, X_test, y_test= wrangle()

In [ ]:
import wrangle2

In [ ]:
train, X_train, y_train, X_validate, y_validate, X_test, y_test= wrangle2.wrangle()

In [ ]:
X_train.head()

In [ ]:
train.shape

In [ ]:
X_train.shape